In [1]:
_RIOT_API_KEY='RGAPI-f133c0d1-25f4-4ab2-ad9e-7d96aa3b73f5'
from riotwatcher import LolWatcher
import helper_functions as help
import role_classification as rc
import analysis_functions as af
import constants
import pandas as pd
import numpy as np
from datetime import datetime
import mysql.connector
from sqlalchemy import create_engine
import concurrent.futures

from collections import Counter
from shapely.geometry import Point
from shapely.geometry.polygon import Polygon

from itertools import compress

lol_watcher = LolWatcher(_RIOT_API_KEY)

In [2]:
#match = lol_watcher.match.by_id('na1', 'NA1_3891774034')
#match = lol_watcher.match.by_id('na1', 'NA1_4720874733')
engine = help.create_mysql_engine()

# Plan for New Role Classification
1. Create new DB table 'lol_match_roles' with: matchId, participantId, Role, validRoles
2. Re-Create function for Jungle Flag
2. Re-Create function for Support Flag
3. Re-Create function for Lane By Posx/Posy
4. Create Mother Function 'determine_roles' To Validate positions for a match. validRoles = True if 1 player in each role, False otherwise. Upload to new 
5. Probably create a 'lol_analysis_view' which pulls needed info from 'lol_match_details', 'lol_match_timeline', and 'lol_match_roles' which an analysis script can pull from.


In [3]:
print(",".join([str(i) for i in [1]]))

1


In [58]:
df = af.prep_comparison_df('souporsecret', engine, minutes_to_analyse = [14], teammates_names = ['irishguy'])
pd.options.display.max_columns=100
df.head(3)

,matchId,championName_x,summonerName_x,role,winFlag_x,teamName_x,teamMate_x,queueName,gameCreationDate,minute,participantId_x,totalGold_x,xp_x,minionsKilled_x,jungleMinionsKilled_x,totalCS_x,totalDamageDoneToChampions_x,championName_y,summonerName_y,winFlag_y,teamName_y,teamMate_y,participantId_y,totalGold_y,xp_y,minionsKilled_y,jungleMinionsKilled_y,totalCS_y,totalDamageDoneToChampions_y,cs_diff,xp_diff,gold_diff,dmg_diff
0,NA1_4630778042,Zyra,souporsecret,SUPPORT,1,Blue,1,5v5 Ranked Flex games,2023-04-15 15:48:49,14,5,3552,3337,11,0,11,4275,Milio,1 tô bún bò hue,0,Red,0,10,2986,3841,1,0,1,994,10,-504,566,3281
1,NA1_4630778042,Draven,aerialblast,BOTTOM,1,Blue,1,5v5 Ranked Flex games,2023-04-15 15:48:49,14,4,4183,4391,100,0,100,841,Ashe,darkness197,0,Red,0,9,4611,4523,104,0,104,4470,-4,-132,-428,-3629
2,NA1_4630778042,KSante,ncm shrek,MIDDLE,1,Blue,1,5v5 Ranked Flex games,2023-04-15 15:48:49,14,3,4210,7146,100,0,100,6404,Gwen,dragonsins2811,0,Red,0,6,4150,6283,89,4,93,6079,7,863,60,325


In [61]:
cols=['role','championName_x','championName_y','winFlag_x','minute','cs_diff','xp_diff','gold_diff','dmg_diff']
mid_df_10 = df[(df.summonerName_x=='irishguy') & (df.role=='JUNGLE') & (df.gameCreationDate > pd.Timestamp('2022-01-01 15:48:49'))][cols]
#  & (df.gameCreationDate > pd.Timestamp('2022-01-01 15:48:49'))

In [62]:

agg_types={'championName_y':'count','winFlag_x':'mean', 'cs_diff':'mean','xp_diff':'mean', 'gold_diff':'mean', 'dmg_diff':'mean' }
rename_dict = {'championName_y':'Games', 'winFlag_x':'Winrate', 'cs_diff':'CS_Diff@14', 'xp_diff':'XP_Diff@14', 'gold_diff':'Gold_Diff@14', 'dmg_diff':'DMG_Diff@14'}
x=mid_df_10.groupby('championName_x', as_index=False,).agg(agg_types).rename(columns=rename_dict).sort_values(by='Games', ascending=False)
format_dict={'Game': "{:.0f}",
                'Winrate': "{:.0%}",
                'CS_Diff@14': "{:.0f}",
                'XP_Diff@14': "{:.0f}",
                'Gold_Diff@14': "{:.0f}",
                'DMG_Diff@14': "{:.0f}"}
return x.style.format(format_dict)

,championName_x,Games,Winrate,CS_Diff@14,XP_Diff@14,Gold_Diff@14,DMG_Diff@14
19,Nautilus,15,80%,-4,-276,-3,-470
23,Pantheon,14,50%,3,173,595,815
33,Trundle,14,64%,0,-75,-74,-510
36,Warwick,13,46%,-17,-657,-723,-1391
30,Shyvana,12,58%,2,-106,-262,-1484
32,Sylas,8,12%,-17,-510,-326,-568
14,Khazix,7,57%,-2,-161,-245,-1201
31,Skarner,7,71%,7,117,215,-586
8,Garen,6,67%,-2,-229,141,-689
7,FiddleSticks,6,50%,-4,-963,-990,-1524


In [77]:
def stats_at_min(summoner_name, role, db_engine, minutes_to_analyse =[14], teammates_names=[],
                 beg_timestamp= pd.Timestamp('2019-01-01 15:48:49'), end_timestamp= pd.Timestamp('2025-12-31 15:48:49'), ):
    df = af.prep_comparison_df(summoner_name, db_engine, minutes_to_analyse = minutes_to_analyse, teammates_names = teammates_names)
    cols=['role','championName_x','championName_y','winFlag_x','minute','cs_diff','xp_diff','gold_diff','dmg_diff']
    mid_df_10 = df[(df.summonerName_x==summoner_name) & (df.role==role) &
                   (df.gameCreationDate > beg_timestamp) &
                   (df.gameCreationDate < end_timestamp)][cols]
    
    agg_types={'championName_y':'count','winFlag_x':'mean', 'cs_diff':'mean','xp_diff':'mean', 'gold_diff':'mean', 'dmg_diff':'mean' }
    rename_dict = {'championName_y':'Games', 'winFlag_x':'Winrate', 'cs_diff':'CS_Diff@14', 'xp_diff':'XP_Diff@14', 'gold_diff':'Gold_Diff@14', 'dmg_diff':'DMG_Diff@14'}
    x=mid_df_10.groupby('championName_x', as_index=False,).agg(agg_types).rename(columns=rename_dict).sort_values(by='Games', ascending=False)
    format_dict={'Game': "{:.0f}",
                    'Winrate': "{:.0%}",
                    'CS_Diff@14': "{:.0f}",
                    'XP_Diff@14': "{:.0f}",
                    'Gold_Diff@14': "{:.0f}",
                    'DMG_Diff@14': "{:.0f}"}
    return x.style.format(format_dict)
stats_at_min('aerialblast', 'SUPPORT', engine, minutes_to_analyse =[14], teammates_names=['souporsecret','SpicedCider','irishguy','radaemon1'])

,championName_x,Games,Winrate,CS_Diff@14,XP_Diff@14,Gold_Diff@14,DMG_Diff@14
9,Nautilus,10,80%,10,39,-283,-44
7,Lux,8,88%,-3,535,868,2028
0,Alistar,4,100%,10,-36,290,-1251
4,Janna,4,100%,-22,-223,-417,-326
1,Amumu,3,100%,5,321,186,29
5,Karma,3,33%,-8,143,396,2647
6,Leona,3,67%,5,-16,-553,-1053
8,Morgana,2,50%,-10,-191,-578,-576
10,Senna,2,50%,-6,-468,-237,-1814
15,Yuumi,2,0%,-20,-1124,-914,-373
